In [7]:
import glob
import numpy as np
import os

In [8]:
from utils.LoRa import MultiBAM

In [9]:
dataset_name_folder = "dataset_new_sf9"
input_layer = 256 * 15
GEENRATE_ = False #Secure accidently running
# Load all .npy files
print("LOAD DATASET")

files = glob.glob(f'{dataset_name_folder}/*.npy')
data_list = [np.load(f) for f in files]

# Flatten each spectrogram to 1D (256*15 = 3840)
X = np.array([d.flatten() for d in data_list])  # shape: (num_samples, 3840)
print(X.shape)
layers = [input_layer, 1024, 256]  # compress 3840 → 1024 → 256
multi_bam = MultiBAM(layers_dims=layers, eta=1e-5)

def train_multibam(model, X, epochs=5, batch_size=32):
    n = len(X)
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        np.random.shuffle(X)
        for i in range(0, n, batch_size):
            batch = X[i:i+batch_size]
            model.train(batch)
        print(f"  -> epoch {epoch+1} done.")

print("TRAIN MULTIBAM")
if (GEENRATE_):
    
    folder_path = "weight"

    # Check if folder exists, if not create it
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder created: {folder_path}")
    else:
        print(f"Folder already exists: {folder_path}")
    train_multibam(multi_bam, X, epochs=10, batch_size=32)
    for i, bam in enumerate(multi_bam.bams):
        np.save(f"{folder_path}/weights_layer_{i}.npy", bam.W)

def load_weight(): 
    ## HOW TO LOAD WEIGHT
    layers = [256*15, 1024, 256] # <-- must match training

    multi_bam = MultiBAM(layers_dims=layers, eta=1e-5)
    for i, bam in enumerate(multi_bam.bams):
        bam.W = np.load(f"weight/weights_layer_{i}.npy")


LOAD DATASET
(0,)
TRAIN MULTIBAM
